## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import GOOGLE_API_KEY

# Configure gmaps API key
gmaps.configure(api_key=GOOGLE_API_KEY)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Saskylakh,RU,71.9167,114.0833,45.61,75,100,9.60,overcast clouds,2022-08-01 04:11:24
1,1,Lieksa,FI,63.3167,30.0167,57.78,85,26,4.68,scattered clouds,2022-08-01 04:11:25
2,2,Vaini,TO,-21.2000,-175.2000,78.96,94,75,6.91,broken clouds,2022-08-01 04:11:26
3,3,Hermanus,ZA,-34.4187,19.2345,38.80,75,0,7.72,clear sky,2022-08-01 04:11:27
4,4,Tromso,NO,69.6496,18.9570,57.38,82,0,0.00,clear sky,2022-08-01 04:11:28


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_min_temp = float(input("What is the minimum temperature you would like for your trip: "))
user_max_temp = float(input("What is the maximum temperature you would like for your trip: "))

What is the minimum temperature you would like for your trip: 40
What is the maximum temperature you would like for your trip: 42


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_temp_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= user_min_temp) & 
                                       (city_data_df["Max Temp"] <= user_max_temp)]

user_temp_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
78,78,Dikson,RU,73.5069,80.5464,41.31,93,100,11.10,overcast clouds,2022-08-01 04:11:05
135,135,Bredasdorp,ZA,-34.5322,20.0403,41.32,85,0,3.96,clear sky,2022-08-01 04:10:01
462,462,Tiksi,RU,71.6872,128.8694,40.08,94,100,17.40,overcast clouds,2022-08-01 04:13:43


In [7]:
# 4a. Determine if there are any empty rows.
user_temp_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_temp_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
78,Dikson,RU,41.31,overcast clouds,73.5069,80.5464,
135,Bredasdorp,ZA,41.32,clear sky,-34.5322,20.0403,
462,Tiksi,RU,40.08,overcast clouds,71.6872,128.8694,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": GOOGLE_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

73.5069
Hotel not found... skipping.
-34.5322
71.6872


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))